In [26]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import time
import os


In [2]:
us_zip_df = pd.read_csv('../Resources/Data/simplemaps_uszips_basicv1.80/uszips.csv')
us_zip_df

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,601,18.18027,-66.75266,Adjuntas,PR,Puerto Rico,True,NaN,16773.0,100.5,72001,Adjuntas,"{""72001"": 98.76, ""72141"": 1.24}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,602,18.36075,-67.17541,Aguada,PR,Puerto Rico,True,NaN,37083.0,472.1,72003,Aguada,"{""72003"": 100}",Aguada,72003,False,False,America/Puerto_Rico
2,603,18.45744,-67.12225,Aguadilla,PR,Puerto Rico,True,NaN,45652.0,513.2,72005,Aguadilla,"{""72005"": 99.76, ""72099"": 0.24}",Aguadilla|Moca,72005|72099,False,False,America/Puerto_Rico
3,606,18.16585,-66.93716,Maricao,PR,Puerto Rico,True,NaN,6231.0,54.3,72093,Maricao,"{""72093"": 82.28, ""72153"": 11.67, ""72121"": 6.05}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,610,18.29110,-67.12243,Anasco,PR,Puerto Rico,True,NaN,26502.0,275.7,72011,Añasco,"{""72011"": 96.71, ""72099"": 2.81, ""72083"": 0.37,...",Añasco|Moca|Las Marías|Aguada,72011|72099|72083|72003,False,False,America/Puerto_Rico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33783,99923,55.98043,-130.03803,Hyder,AK,Alaska,True,NaN,12.0,0.3,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33784,99925,55.55398,-132.96276,Klawock,AK,Alaska,True,NaN,990.0,7.1,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka
33785,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,True,NaN,1582.0,4.6,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Metlakatla
33786,99927,56.33305,-133.60044,Point Baker,AK,Alaska,True,NaN,0.0,0.0,2198,Prince of Wales-Hyder,"{""02198"": 100}",Prince of Wales-Hyder,02198,False,False,America/Sitka


In [21]:
states_list = list(us_zip_df.state_id.unique().ravel())


['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']


In [23]:
print("Here are the state ID's:")
print(sorted(states_list))
user_state = str(input('Please provide the state id(EX: AL for Alabama): '))


Here are the state ID's:
['AK', 'AL', 'AR', 'AS', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'GU', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MP', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VI', 'VT', 'WA', 'WI', 'WV', 'WY']
Please provide the state id(EX: AL for Alabama): AL


In [24]:
state_zips_df = us_zip_df[us_zip_df.state_id == user_state] 

In [25]:
state_zips_df

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
11492,35004,33.60369,-86.49333,Moody,AL,Alabama,True,NaN,11944.0,267.5,1115,St. Clair,"{""01115"": 100}",St. Clair,01115,False,False,America/Chicago
11493,35005,33.59215,-86.99833,Adamsville,AL,Alabama,True,NaN,8815.0,107.2,1073,Jefferson,"{""01073"": 100}",Jefferson,01073,False,False,America/Chicago
11494,35006,33.43159,-87.19961,Adger,AL,Alabama,True,NaN,3170.0,11.8,1073,Jefferson,"{""01073"": 95.12, ""01127"": 2.88, ""01125"": 2.0}",Jefferson|Walker|Tuscaloosa,01073|01127|01125,False,False,America/Chicago
11495,35007,33.21751,-86.79307,Alabaster,AL,Alabama,True,NaN,26510.0,262.1,1117,Shelby,"{""01117"": 100}",Shelby,01117,False,False,America/Chicago
11496,35010,32.91433,-85.93816,Alexander City,AL,Alabama,True,NaN,20208.0,35.0,1123,Tallapoosa,"{""01123"": 75.54, ""01037"": 18.22, ""01051"": 6.24}",Tallapoosa|Coosa|Elmore,01123|01037|01051,False,False,America/Chicago
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12143,36916,32.23788,-88.04143,Pennington,AL,Alabama,True,NaN,1030.0,8.9,1023,Choctaw,"{""01023"": 100}",Choctaw,01023,False,False,America/Chicago
12144,36919,31.76765,-88.28940,Silas,AL,Alabama,True,NaN,1707.0,4.3,1023,Choctaw,"{""01023"": 100}",Choctaw,01023,False,False,America/Chicago
12145,36921,31.96956,-88.19108,Toxey,AL,Alabama,True,NaN,814.0,3.2,1023,Choctaw,"{""01023"": 100}",Choctaw,01023,False,False,America/Chicago
12146,36922,32.30061,-88.15068,Ward,AL,Alabama,True,NaN,651.0,3.2,1023,Choctaw,"{""01023"": 51.52, ""01119"": 48.48}",Choctaw|Sumter,01023|01119,False,False,America/Chicago


In [30]:
# Leaf directory
directory = user_state
 
# Parent Directories
parent_dir = "../Resources/Data/user_scrape_data/"
 
# Path
path = os.path.join(parent_dir, directory)
 
# Create the directory
# By setting exist_ok as True
# error caused due already
# existing directory can be suppressed
# but other OSError may be raised
# due to other error like
# invalid path name
try:
    os.makedirs(path, exist_ok = True)
    print("Directory '%s' created successfully" %path)
except OSError as error:
    print("Directory '%s' can not be created")



Directory '../Resources/Data/user_scrape_data/AL' created successfully


In [31]:
state_zips_df.isnull().sum()

zip                   0
lat                   0
lng                   0
city                  0
state_id              0
state_name            0
zcta                  0
parent_zcta         656
population           22
density              22
county_fips           0
county_name           0
county_weights        0
county_names_all      0
county_fips_all       0
imprecise             0
military              0
timezone              0
dtype: int64

In [37]:
state_path_zip_output = os.path.join(path,'zips.csv')
state_path_zip_output
state_zips_df.to_csv(state_path_zip_output, index=False)

In [2]:
zip_csv = '../Resources/Data/usa_zipcode_of_or-1554j.csv'
zip_df = pd.read_csv(zip_csv)

In [3]:
zip_df.dropna(inplace=True)
zip_df.tail()

,zip,Zipcode name,City,State,County Name
457,97497,"WOLF CREEK, OR",WOLF CREEK,OR,JOSEPHINE
458,97071,"WOODBURN, OR",WOODBURN,OR,MARION
459,97498,"YACHATS, OR",YACHATS,OR,LINCOLN
460,97148,"YAMHILL, OR",YAMHILL,OR,YAMHILL
461,97499,"YONCALLA, OR",YONCALLA,OR,DOUGLAS


In [4]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [5]:
zip_list = zip_df.zip.to_list()
zip_list

['97810',
 '97901',
 '97406',
 '97321',
 '97322',
 '97407',
 '97324',
 '97409',
 '97101',
 '97001',
 '97102',
 '97812',
 '97520',
 '97103',
 '97813',
 '97325',
 '97002',
 '97410',
 '97814',
 '97411',
 '97106',
 '97817',
 '97107',
 '97621',
 '97108',
 '97004',
 '97003',
 '97005',
 '97006',
 '97007',
 '97008',
 '97075',
 '97076',
 '97077',
 '97078',
 '97701',
 '97702',
 '97707',
 '97708',
 '97709',
 '97412',
 '97326',
 '97413',
 '97622',
 '97818',
 '97623',
 '97009',
 '97010',
 '97819',
 '97011',
 '97414',
 '97903',
 '97415',
 '97327',
 '97720',
 '97522',
 '97109',
 '97416',
 '97730',
 '97013',
 '97110',
 '97820',
 '97417',
 '97111',
 '97014',
 '97329',
 '97523',
 '97502',
 '97731',
 '97419',
 '97624',
 '97641',
 '97015',
 '97016',
 '97112',
 '97017',
 '97018',
 '97823',
 '97420',
 '97423',
 '97019',
 '97113',
 '97330',
 '97331',
 '97333',
 '97339',
 '97424',
 '97824',
 '97335',
 '97732',
 '97604',
 '97336',
 '97733',
 '97426',
 '97734',
 '97625',
 '97338',
 '97089',
 '97429',
 '97114',


In [7]:
driver = webdriver.Chrome()


In [10]:
json_list = []
df = pd.DataFrame()
for zip_group in chunker(zip_list,8):
    print(zip_group)
    driver.get("https://www.ewg.org/tapwater/")
    #time.sleep(2)
    for zipcode in zip_group:
        print(zipcode)
        #try:
        #driver.manage().timeouts().pageLoadTimeout(40, TimeUnit.SECONDS)

        element = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[1]')
        element.clear()
        element.send_keys(zipcode)
        button = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[3]').click()
        #time.sleep(2)
        zip_url = driver.current_url
        url = requests.get(zip_url)
        table = pd.read_html(url.text)
        data = table[0]
        data['Zip'] = zipcode
        df = df.append(data)
        
        df.to_csv('../Resources/Data/Utilities_zipcode.csv', index=False)
        
        #Navigate back to the home page after finishing the zip code scrape    
        driver.back()
        #time.sleep(2)

['97810', '97901', '97406', '97321', '97322', '97407', '97324', '97409']
97810
97901
97406
97321


ValueError: No tables found

In [36]:
driver = webdriver.Chrome()

driver.get("https://www.ewg.org/tapwater/")
time.sleep(2)
element = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[1]')
element.clear()
element.send_keys('97045')
button = driver.find_element(By.XPATH,'/html/body/main/section[1]/form/input[3]').click()
time.sleep(2)
zip_url = driver.current_url
url = requests.get(zip_url)
table = pd.read_html(url.text)
data = table[0]


#Navigate back to the home page after finishing the zip code scrape    
driver.back()
time.sleep(2)

In [37]:
data

,Utility name,City,People served
0,North Clackamas County Water Commission,"Oregon City, OR","Population served:87,700"
1,South Fork Water Board,"Oregon City, OR","Population served:65,000"
2,Sunrise Water Authority,"Happy Valley, OR","Population served:46,228"
3,Clackamas River Water - Clackamas,"Clackamas, OR","Population served:41,338"
4,Lake Oswego Municipal Water,"Lake Oswego, OR","Population served:36,453"
5,Oregon City,"Oregon City, OR","Population served:33,940"
6,Oak Lodge Water District,"Milwaukie, OR","Population served:30,000"
7,City of West Linn,"West Linn, OR","Population served:25,002"
8,City of Milwaukie,"Milwaukie, OR","Population served:20,500"
9,City of Wilsonville,"Wilsonville, OR","Population served:22,729"


In [39]:
data['Zip'] = '97045'
#results = table[0].to_json()
#json_list.append(results)
#print(json_list)
data

,Utility name,City,People served,Zip
0,North Clackamas County Water Commission,"Oregon City, OR","Population served:87,700",97045
1,South Fork Water Board,"Oregon City, OR","Population served:65,000",97045
2,Sunrise Water Authority,"Happy Valley, OR","Population served:46,228",97045
3,Clackamas River Water - Clackamas,"Clackamas, OR","Population served:41,338",97045
4,Lake Oswego Municipal Water,"Lake Oswego, OR","Population served:36,453",97045
5,Oregon City,"Oregon City, OR","Population served:33,940",97045
6,Oak Lodge Water District,"Milwaukie, OR","Population served:30,000",97045
7,City of West Linn,"West Linn, OR","Population served:25,002",97045
8,City of Milwaukie,"Milwaukie, OR","Population served:20,500",97045
9,City of Wilsonville,"Wilsonville, OR","Population served:22,729",97045


In [41]:
df = df.append(data)
df

,Utility name,City,People served,Zip
0,North Clackamas County Water Commission,"Oregon City, OR","Population served:87,700",97045
1,South Fork Water Board,"Oregon City, OR","Population served:65,000",97045
2,Sunrise Water Authority,"Happy Valley, OR","Population served:46,228",97045
3,Clackamas River Water - Clackamas,"Clackamas, OR","Population served:41,338",97045
4,Lake Oswego Municipal Water,"Lake Oswego, OR","Population served:36,453",97045
5,Oregon City,"Oregon City, OR","Population served:33,940",97045
6,Oak Lodge Water District,"Milwaukie, OR","Population served:30,000",97045
7,City of West Linn,"West Linn, OR","Population served:25,002",97045
8,City of Milwaukie,"Milwaukie, OR","Population served:20,500",97045
9,City of Wilsonville,"Wilsonville, OR","Population served:22,729",97045


In [24]:
#df.set_index(zipcode)
df.to_csv('../Resources/Data/Utilities_zipcode.csv')

In [25]:
data


'{"Utility name":{"0":"City of Pendleton","1":"City of Hermiston","2":"Milton-freewater, City Of","3":"Umatilla, City of","4":"Stanfield, City of"},"City":{"0":"Pendleton, OR","1":"Hermiston, OR","2":"Milton-freewater, OR","3":"Umatilla, OR","4":"Stanfield, OR"},"People served":{"0":"Population served:17,310","1":"Population served:18,200","2":"Population served:7,070","3":"Population served:6,980","4":"Population served:2,115"}}'

In [28]:
data = dict(data)

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [43]:
df

,Utility name,City,People served,Zip
0,City of Pendleton,"Pendleton, OR","Population served:17,310",97810
1,City of Hermiston,"Hermiston, OR","Population served:18,200",97810
2,"Milton-freewater, City Of","Milton-freewater, OR","Population served:7,070",97810
3,"Umatilla, City of","Umatilla, OR","Population served:6,980",97810
4,"Stanfield, City of","Stanfield, OR","Population served:2,115",97810
0,"Adrian, City of",Adrian,Population served: 180,97901
1,City of Ontario,Ontario,"Population served: 14,465",97901
2,Fischers Place Mobile Home Park,Scappoose,Population served: 125,97901
3,"Jordan Valley, City of",Jordan Valley,Population served: 175,97901
4,"Nyssa, City of",Nyssa,"Population served: 3,285",97901


In [38]:
zip_list = state_zips_df.zip.to_list()

In [39]:
zip_list

[35004,
 35005,
 35006,
 35007,
 35010,
 35013,
 35014,
 35016,
 35019,
 35020,
 35022,
 35023,
 35031,
 35032,
 35033,
 35034,
 35035,
 35036,
 35040,
 35042,
 35043,
 35044,
 35045,
 35046,
 35049,
 35051,
 35052,
 35053,
 35054,
 35055,
 35057,
 35058,
 35060,
 35061,
 35062,
 35063,
 35064,
 35068,
 35070,
 35071,
 35072,
 35073,
 35074,
 35077,
 35078,
 35079,
 35080,
 35082,
 35083,
 35085,
 35087,
 35089,
 35091,
 35094,
 35096,
 35097,
 35098,
 35111,
 35112,
 35114,
 35115,
 35116,
 35117,
 35118,
 35119,
 35120,
 35121,
 35124,
 35125,
 35126,
 35127,
 35128,
 35130,
 35131,
 35133,
 35135,
 35136,
 35143,
 35146,
 35147,
 35148,
 35149,
 35150,
 35151,
 35160,
 35171,
 35172,
 35173,
 35175,
 35176,
 35178,
 35179,
 35180,
 35181,
 35183,
 35184,
 35185,
 35186,
 35188,
 35203,
 35204,
 35205,
 35206,
 35207,
 35208,
 35209,
 35210,
 35211,
 35212,
 35213,
 35214,
 35215,
 35216,
 35217,
 35218,
 35221,
 35222,
 35223,
 35224,
 35226,
 35228,
 35229,
 35233,
 35234,
 35235,


In [40]:
temp_list = zip_list[:5]
temp_list

[35004, 35005, 35006, 35007, 35010]

In [47]:
utilities_df = pd.DataFrame()
for zipcode in temp_list:
    page="https://www.ewg.org/tapwater/search-results.php?zip5="+ str(zipcode)+"&searchtype=zip"
    url = requests.get(page)
    
    try:
        table = pd.read_html(url.text)
        data = table[0]
        data['Zip'] = zipcode
        utilities_df = df.append(data)
    except ValueError:
        print(f'No systems found that match your search for: {zipcode}')
   # print(page)

No systems found that match your search for: 35004


In [48]:
utilities_df

,Utility name,City,People served,Zip
0,Bessemer Water Service,Bessemer,"Population served: 85,113",35005
1,Birmingham Water Works Board,Birmingham,"Population served: 585,000",35005
2,Brookside Water Works,Brookside,"Population served: 2,550",35005
3,Governmental Utility Service Corporation,Bessemer,Population served: 32,35005
4,Graysville Water & Sewer Board,Graysville,"Population served: 9,144",35005
...,...,...,...,...
21,Tallassee Water Works,Tallassee,"Population served: 7,410",35010
22,Tri Community Water System,Millbrook,"Population served: 11,505",35010
23,Wall Street Water Authority,Tallassee,"Population served: 5,709",35010
24,Walnut Hill Water Authority,Dadeville,"Population served: 11,304",35010


In [49]:
state_path_utility_output = os.path.join(path,'utilities.csv')
utilities_df.to_csv(state_path_utility_output, index=False)


In [51]:
utilities_df['Utility name'].value_counts()

Bessemer Water Service                         2
Jacksons Gap Water Authority                   2
Camp Hill Utilities Board                      2
Central Elmore Water Authority                 2
Dadeville Water Works & Sewage Board           2
Daviston Water System, Town of                 2
Eclectic Water Works & Sewer Board             2
Elmore Water and Sewer Authority               2
Five Star Water Supply District                2
Friendship Water Works                         2
Goodwater Water Works and Sewer Bd, City of    2
Hackneyville Water & Fire Pro Authority        2
Holtville Water System                         2
Kellyton Water System                          2
Birmingham Water Works Board                   2
Lake Mitchell Water, Incorporated              2
Millbrook Utilities                            2
New Site Water System                          2
Parker Creek Water Company                     2
Ray Community Water & Fire Pro Authority       2
Rockford Water Works

In [52]:
utilities_df.drop_duplicates(subset=['Utility name'],inplace=True)

In [54]:
utilities_df['Utility name'].value_counts()

Bessemer Water Service                         1
Jacksons Gap Water Authority                   1
Camp Hill Utilities Board                      1
Central Elmore Water Authority                 1
Dadeville Water Works & Sewage Board           1
Daviston Water System, Town of                 1
Eclectic Water Works & Sewer Board             1
Elmore Water and Sewer Authority               1
Five Star Water Supply District                1
Friendship Water Works                         1
Goodwater Water Works and Sewer Bd, City of    1
Hackneyville Water & Fire Pro Authority        1
Holtville Water System                         1
Kellyton Water System                          1
Wilton Water Works                             1
Lake Mitchell Water, Incorporated              1
Millbrook Utilities                            1
New Site Water System                          1
Parker Creek Water Company                     1
Ray Community Water & Fire Pro Authority       1
Rockford Water Works

In [55]:
utilities_df.isnull().sum()

Utility name     0
City             0
People served    0
Zip              0
dtype: int64

In [14]:
df.to_csv('../Resources/Data/Raw_Utilities_zipcode.csv')

In [16]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

df['Utility name'].value_counts()

Portland Water Bureau                                 62
Cty of Gresham                                        62
Rockwood People's Utilities Department                62
Lorna Portland Water LLC                              43
Sigler Cove Marina                                    43
City of Troutdale                                     43
Corbett Water District                                43
Fairview, City of                                     43
Interlachen Water People's Utility District           43
Burlington Water District                             43
Lusted Water District                                 43
Mayfair Moorage Inc.                                  43
Palatine Hill Water District                          43
Rocky Pointe Marina                                   43
Pleasant Home Water District                          43
Wapato Moorage Association                            43
Sandy Boulevard Mobile Villa                          43
Sauvie Island Moorage          

In [35]:
clean_df = df.drop_duplicates(subset=['Utility name'])

In [38]:
clean_df.shape

(419, 4)

In [39]:
clean_df = clean_df.dropna(axis=0)

In [40]:
clean_df.isnull().values.any()


False

In [41]:
clean_df.reset_index(drop=True)

,Utility name,City,People served,Zip
0,City of Pendleton,"Pendleton, OR","Population served:17,310",97810
1,City of Hermiston,"Hermiston, OR","Population served:18,200",97810
2,"Milton-freewater, City Of","Milton-freewater, OR","Population served:7,070",97810
3,"Umatilla, City of","Umatilla, OR","Population served:6,980",97810
4,"Stanfield, City of","Stanfield, OR","Population served:2,115",97810
5,"Adrian, City of",Adrian,Population served: 180,97901
6,City of Ontario,Ontario,"Population served: 14,465",97901
7,Fischers Place Mobile Home Park,Scappoose,Population served: 125,97901
8,"Jordan Valley, City of",Jordan Valley,Population served: 175,97901
9,"Nyssa, City of",Nyssa,"Population served: 3,285",97901


In [42]:
clean_df['Utility name'].value_counts()

City of Pendleton                                     1
Willow Estates Mobile Home Park                       1
Sigler Cove Marina                                    1
Palatine Hill Water District                          1
Mayfair Moorage Inc.                                  1
Lusted Water District                                 1
Lorna Portland Water LLC                              1
Interlachen Water People's Utility District           1
Fairview, City of                                     1
Corbett Water District                                1
City of Troutdale                                     1
Manna House Church                                    1
Wapato Moorage Association                            1
Burlington Water District                             1
Rockwood People's Utilities Department                1
Cty of Gresham                                        1
Portland Water Bureau                                 1
Lexington, City Of                              

In [92]:
clean_df.to_csv('../Resources/Data/Clean_Utilities_zipcode.csv')

In [44]:
utility_list = clean_df['Utility name'].to_list()
utility_list

['City of Pendleton',
 'City of Hermiston',
 'Milton-freewater, City Of',
 'Umatilla, City of',
 'Stanfield, City of',
 'Adrian, City of',
 'City of Ontario',
 'Fischers Place Mobile Home Park',
 'Jordan Valley, City of',
 'Nyssa, City of',
 'River Village Mobile Home Park',
 'Vale, City of',
 'Brookings, City of',
 'Gold Beach, City of',
 "Harbor Water People's Utility District",
 'Langlois Water District',
 'Nesika Beach-ophir Water District',
 'Old Sheep Ranch Water Association',
 'Port Orford, City of',
 'Rainbow Rock Service Association',
 'Rainbow Rock Village Mobile Home Park',
 'Saunders Creek Hoc',
 'City of Corvallis',
 'City of Albany',
 'City of Lebanon',
 'City of Dallas',
 'Monmouth, City of',
 'Sweet Home, City of',
 'Independence Water System',
 'Philomath Public Works',
 'Harrisburg, City of',
 'Luckiamute Domestic Water Co-op',
 'Lyons Mehama Water District',
 'Bandon, City of',
 'Bridge Water District',
 'Coos Bay North Bend Water Board',
 'Coquille, City of',
 'Coun

In [46]:
driver = webdriver.Chrome()
driver.get("https://www.ewg.org/tapwater/advanced-search.php")

In [47]:
utility_input = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[1]')

In [48]:
utility_input.clear()

In [49]:
utility_input.send_keys(utility_list[0])

In [50]:
go_btn = driver.find_element(By.XPATH, '/html/body/div[3]/main/div/section[1]/form/input[3]')

In [51]:
go_btn.click()

In [52]:
utility = driver.find_element(By.XPATH, '/html/body/div[3]/main/figure/table/tbody/tr/td[1]/a')

In [53]:
utility.click()

In [82]:
page_source = driver.page_source

html_soup = soup(page_source, 'html.parser')
contaminant_list = []
#Get the name of the Water Utility
Utility = html_soup.find('h1').text
Utility


#get the html data we need
data_box = html_soup.find_all('div', class_='contaminant-name')

for i in range(len(data_box)):
    data = data_box[i].find_all('span')
    data_measure = []
    d = {
        'Utility' : Utility,
        'Contaminant': '', 
        'Utility Measuremnt':'', 
        'EWG HEALTH GUIDELINE': '',
        'Legal Limit':'' 
    }

    contaminant_name = data_box[i].find('h3')
    d['Contaminant'] = contaminant_name

    for j in range(len(data)):
        measurement = data[j].text
        #print(measurement)
        data_measure.append(measurement)
        #print(data_measure)

    try:
        d['Utility Measuremnt'] = data_measure[data_measure.index('THIS UTILITY')+1]
    except ValueError:
        print("A value error arose")
    except:
        print("Something else went wrong")
    try:
        d['EWG HEALTH GUIDELINE'] = data_measure[data_measure.index('EWG HEALTH GUIDELINE')+1]
    except ValueError:
        print("A value error arose")
    except:
        print("Something else went wrong") 
    try:
        d['Legal Limit'] = data_measure[data_measure.index('LEGAL LIMIT')+1]
    except ValueError:
        print("A value error arose")
    except:
        print("Something else went wrong") 

    contaminant_list.append(d)

A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose
A value error arose


In [65]:
data

[<span class="potentital-effect"> Potential Effect: cancer</span>,
 <span class="detect-times-greater-than">188x</span>,
 <span class="potentital-effect"> EWG'S HEALTH GUIDELINE</span>,
 <span>THIS UTILITY</span>,
 <span>0.750 ppb</span>,
 <span>EWG HEALTH GUIDELINE</span>,
 <span>0.004 ppb</span>,
 <span>LEGAL LIMIT</span>,
 <span>10 ppb</span>]

In [83]:
df1 = pd.DataFrame(contaminant_list)

In [84]:
df1

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,City of Pendleton,[Arsenic],0.750 ppb,0.004 ppb,10 ppb
1,City of Pendleton,[Chromium (hexavalent)],0.0472 ppb,0.02 ppb,
2,City of Pendleton,[Haloacetic acids (HAA5)†],15.3 ppb,0.1 ppb,60 ppb
3,City of Pendleton,[Haloacetic acids (HAA9)†],16.2 ppb,0.06 ppb,
4,City of Pendleton,[Nitrate],1.40 ppm,0.14 ppm,10 ppm
5,City of Pendleton,[Total trihalomethanes (TTHMs)†],25.5 ppb,0.15 ppb,80 ppb
6,City of Pendleton,[Uranium],0.89 pCi/L,0.43 pCi/L,20 pCi/L
7,City of Pendleton,[Chlorate],125.9 ppb,210 ppb,
8,City of Pendleton,[Chromium (total)],0.0338 ppb,,100 ppb
9,City of Pendleton,[Manganese],3.99 ppb,100 ppb,


In [67]:
list_cont = df1.Contaminant.to_list()
list_cont

[<h3>Arsenic</h3>,
 <h3>Chromium (hexavalent)</h3>,
 <h3>Haloacetic acids (HAA5)†</h3>,
 <h3>Haloacetic acids (HAA9)†</h3>,
 <h3>Nitrate</h3>,
 <h3>Total trihalomethanes (TTHMs)†</h3>,
 <h3>Uranium</h3>,
 <h3>Chlorate</h3>,
 <h3>Chromium (total)</h3>,
 <h3>Manganese</h3>,
 <h3>Molybdenum</h3>,
 <h3>Strontium</h3>,
 <h3>Vanadium</h3>,
 None]

In [69]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

In [71]:
text = cleanhtml(str(list_cont[0]))

In [72]:
text

'Arsenic'

In [74]:
df1.dtypes

Utility                 object
Contaminant             object
Utility Measuremnt      object
EWG HEALTH GUIDELINE    object
Legal Limit             object
dtype: object

In [86]:
df1 = df1.astype({"Contaminant": str})

In [87]:
df1.dtypes

Utility                 object
Contaminant             object
Utility Measuremnt      object
EWG HEALTH GUIDELINE    object
Legal Limit             object
dtype: object

In [90]:
df1.Contaminant = df1.Contaminant.apply(lambda x: cleanhtml(x))

In [91]:
df1

,Utility,Contaminant,Utility Measuremnt,EWG HEALTH GUIDELINE,Legal Limit
0,City of Pendleton,Arsenic,0.750 ppb,0.004 ppb,10 ppb
1,City of Pendleton,Chromium (hexavalent),0.0472 ppb,0.02 ppb,
2,City of Pendleton,Haloacetic acids (HAA5)†,15.3 ppb,0.1 ppb,60 ppb
3,City of Pendleton,Haloacetic acids (HAA9)†,16.2 ppb,0.06 ppb,
4,City of Pendleton,Nitrate,1.40 ppm,0.14 ppm,10 ppm
5,City of Pendleton,Total trihalomethanes (TTHMs)†,25.5 ppb,0.15 ppb,80 ppb
6,City of Pendleton,Uranium,0.89 pCi/L,0.43 pCi/L,20 pCi/L
7,City of Pendleton,Chlorate,125.9 ppb,210 ppb,
8,City of Pendleton,Chromium (total),0.0338 ppb,,100 ppb
9,City of Pendleton,Manganese,3.99 ppb,100 ppb,


In [ ]:
us_zips_df = pd.read